# Deep Dive: SEC Financial Analysis of Nursing Home REITs

## Comprehensive Comparison: OHI vs CTRE vs SBRA

This notebook provides a detailed financial analysis of three skilled nursing facility REITs using SEC data:
- **OHI**: Omega Healthcare Investors
- **CTRE**: CareTrust REIT
- **SBRA**: Sabra Health Care REIT

### Analysis Framework:
1. **Scale & Size**: Total assets, property portfolio size
2. **Revenue Analysis**: Growth trends, stability, seasonality
3. **Profitability**: NOI margins, net income trends
4. **Balance Sheet Health**: Leverage, debt maturity, equity
5. **Growth Profile**: Historical growth rates, momentum
6. **Risk Assessment**: Volatility, leverage risk, concentration
7. **Valuation Metrics**: Key ratios and comparative analysis

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print("✅ Libraries loaded successfully")

✅ Libraries loaded successfully


## 1. Data Loading & Preparation

In [2]:
# Load all SEC data for the three REITs
data_path = Path('../../data/processed')

# Define the metrics we want to analyze
metrics = {
    'revenues': 'Revenues',
    'assets': 'Total Assets',
    'liabilities': 'Total Liabilities',
    'stockholdersequity': 'Stockholders Equity',
    'netincomeloss': 'Net Income (Loss)',
    'realestateinvestmentpropertynet': 'Real Estate Investment Property',
    'longtermdebt': 'Long-term Debt',
    'operatingincomeloss': 'Operating Income (Loss)'
}

# Load data for each REIT
reits = ['ohi', 'ctre', 'sbra']
reit_names = {
    'ohi': 'Omega Healthcare (OHI)',
    'ctre': 'CareTrust REIT (CTRE)',
    'sbra': 'Sabra Healthcare (SBRA)'
}

# Dictionary to store all data
reit_data = {}

for reit in reits:
    reit_data[reit] = {}
    for metric_key, metric_name in metrics.items():
        file_name = f'sec_{reit}_{metric_key}_latest.csv'
        file_path = data_path / file_name
        
        if file_path.exists():
            df = pd.read_csv(file_path)
            df['end'] = pd.to_datetime(df['end'])
            df = df.sort_values('end', ascending=False)
            reit_data[reit][metric_key] = df
            print(f"✅ Loaded {metric_name} for {reit.upper()}: {len(df)} records")
        else:
            print(f"⚠️  Missing {metric_name} for {reit.upper()}")
            reit_data[reit][metric_key] = pd.DataFrame()

print("\n✅ All data loaded successfully")

✅ Loaded Revenues for OHI: 103 records
✅ Loaded Total Assets for OHI: 122 records
✅ Loaded Total Liabilities for OHI: 122 records
✅ Loaded Stockholders Equity for OHI: 134 records
✅ Loaded Net Income (Loss) for OHI: 112 records
✅ Loaded Real Estate Investment Property for OHI: 122 records
✅ Loaded Long-term Debt for OHI: 106 records
✅ Loaded Operating Income (Loss) for OHI: 138 records
✅ Loaded Revenues for CTRE: 203 records
✅ Loaded Total Assets for CTRE: 90 records
✅ Loaded Total Liabilities for CTRE: 90 records
✅ Loaded Stockholders Equity for CTRE: 174 records
✅ Loaded Net Income (Loss) for CTRE: 241 records
✅ Loaded Real Estate Investment Property for CTRE: 90 records
✅ Loaded Long-term Debt for CTRE: 90 records
⚠️  Missing Operating Income (Loss) for CTRE
✅ Loaded Revenues for SBRA: 181 records
✅ Loaded Total Assets for SBRA: 115 records
✅ Loaded Total Liabilities for SBRA: 115 records
✅ Loaded Stockholders Equity for SBRA: 179 records
✅ Loaded Net Income (Loss) for SBRA: 169 rec

## 2. Scale & Size Comparison

Let's start by understanding the relative size of each REIT.

In [3]:
# Get latest metrics for each REIT
size_metrics = []

for reit in reits:
    # Get latest assets
    assets_df = reit_data[reit]['assets']
    if len(assets_df) > 0:
        latest_assets = assets_df.iloc[0]['val']
    else:
        latest_assets = None
    
    # Get latest real estate property
    re_df = reit_data[reit]['realestateinvestmentpropertynet']
    if len(re_df) > 0:
        latest_re = re_df.iloc[0]['val']
    else:
        latest_re = None
    
    # Get latest revenue (TTM)
    rev_df = reit_data[reit]['revenues']
    quarterly = rev_df[rev_df['fp'].str.startswith('Q')].copy() if len(rev_df) > 0 else pd.DataFrame()
    if len(quarterly) >= 4:
        ttm_revenue = quarterly.iloc[0:4]['val'].sum()
    else:
        ttm_revenue = None
    
    size_metrics.append({
        'REIT': reit.upper(),
        'Name': reit_names[reit],
        'Total Assets ($B)': latest_assets / 1e9 if latest_assets else None,
        'Real Estate ($B)': latest_re / 1e9 if latest_re else None,
        'TTM Revenue ($M)': ttm_revenue / 1e6 if ttm_revenue else None
    })

size_df = pd.DataFrame(size_metrics)
size_df

,REIT,Name,Total Assets ($B),Real Estate ($B),TTM Revenue ($M)
0,OHI,Omega Healthcare (OHI),10.55,6.80,"1,394.61"
1,CTRE,CareTrust REIT (CTRE),4.66,3.26,495.56
2,SBRA,Sabra Healthcare (SBRA),5.33,4.48,923.39


In [4]:
# Visualize size comparison
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Total Assets', 'Real Estate Portfolio', 'TTM Revenue'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]]
)

# Assets
fig.add_trace(
    go.Bar(x=size_df['REIT'], y=size_df['Total Assets ($B)'], name='Total Assets', marker_color='#1f77b4'),
    row=1, col=1
)

# Real Estate
fig.add_trace(
    go.Bar(x=size_df['REIT'], y=size_df['Real Estate ($B)'], name='Real Estate', marker_color='#2ca02c'),
    row=1, col=2
)

# Revenue
fig.add_trace(
    go.Bar(x=size_df['REIT'], y=size_df['TTM Revenue ($M)'], name='TTM Revenue', marker_color='#ff7f0e'),
    row=1, col=3
)

fig.update_layout(height=400, showlegend=False, title_text="REIT Size Comparison")
fig.update_yaxes(title_text="Billions USD", row=1, col=1)
fig.update_yaxes(title_text="Billions USD", row=1, col=2)
fig.update_yaxes(title_text="Millions USD", row=1, col=3)

fig.show()

# Key insights
largest_assets = size_df.loc[size_df['Total Assets ($B)'].idxmax(), 'REIT']
largest_revenue = size_df.loc[size_df['TTM Revenue ($M)'].idxmax(), 'REIT']

print(f"\n📊 Key Insights:")
print(f"Largest by Assets: {largest_assets}")
print(f"Largest by Revenue: {largest_revenue}")
print(f"\nOHI is {size_df[size_df['REIT']=='OHI']['Total Assets ($B)'].values[0] / size_df[size_df['REIT']=='CTRE']['Total Assets ($B)'].values[0]:.1f}x the size of CTRE by assets")


📊 Key Insights:
Largest by Assets: OHI
Largest by Revenue: OHI

OHI is 2.3x the size of CTRE by assets


## 3. Revenue Analysis

Deep dive into revenue trends, growth rates, and stability.

In [5]:
# Plot quarterly revenue trends for all three REITs
fig = go.Figure()

colors = {'ohi': '#1f77b4', 'ctre': '#ff7f0e', 'sbra': '#2ca02c'}

for reit in reits:
    rev_df = reit_data[reit]['revenues']
    quarterly = rev_df[rev_df['fp'].str.startswith('Q')].copy()
    quarterly = quarterly.sort_values('end')
    
    fig.add_trace(go.Scatter(
        x=quarterly['end'],
        y=quarterly['val'] / 1e6,
        mode='lines+markers',
        name=reit.upper(),
        line=dict(width=3, color=colors[reit]),
        marker=dict(size=6)
    ))

fig.update_layout(
    title='Quarterly Revenue Trends (2010-2025)',
    xaxis_title='Quarter End Date',
    yaxis_title='Revenue (Millions USD)',
    hovermode='x unified',
    height=500
)

fig.show()

In [6]:
# Calculate growth rates
growth_analysis = []

for reit in reits:
    rev_df = reit_data[reit]['revenues']
    quarterly = rev_df[rev_df['fp'].str.startswith('Q')].copy()
    quarterly = quarterly.sort_values('end', ascending=False)
    
    if len(quarterly) >= 8:
        # Latest 4 quarters (TTM)
        latest_ttm = quarterly.iloc[0:4]['val'].sum()
        
        # Prior 4 quarters
        prior_ttm = quarterly.iloc[4:8]['val'].sum()
        
        # YoY growth
        yoy_growth = ((latest_ttm / prior_ttm) - 1) * 100
        
        # Calculate 3-year CAGR
        if len(quarterly) >= 16:
            ttm_3y_ago = quarterly.iloc[12:16]['val'].sum()
            cagr_3y = (((latest_ttm / ttm_3y_ago) ** (1/3)) - 1) * 100
        else:
            cagr_3y = None
        
        # Calculate 5-year CAGR
        if len(quarterly) >= 24:
            ttm_5y_ago = quarterly.iloc[20:24]['val'].sum()
            cagr_5y = (((latest_ttm / ttm_5y_ago) ** (1/5)) - 1) * 100
        else:
            cagr_5y = None
        
        # Calculate revenue volatility (std dev of QoQ growth)
        quarterly['qoq_growth'] = quarterly['val'].pct_change(-1) * 100
        volatility = quarterly['qoq_growth'].std()
        
        growth_analysis.append({
            'REIT': reit.upper(),
            'TTM Revenue ($M)': latest_ttm / 1e6,
            'YoY Growth (%)': yoy_growth,
            '3Y CAGR (%)': cagr_3y,
            '5Y CAGR (%)': cagr_5y,
            'Revenue Volatility (%)': volatility
        })

growth_df = pd.DataFrame(growth_analysis)
growth_df

,REIT,TTM Revenue ($M),YoY Growth (%),3Y CAGR (%),5Y CAGR (%),Revenue Volatility (%)
0,OHI,"1,394.61",-21.37,-13.18,-0.51,84.84
1,CTRE,627.52,80.78,24.99,16.25,80.33
2,SBRA,923.39,-24.07,-14.01,0.45,73.36


In [7]:
# Visualize growth comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Revenue Growth Rates', 'Revenue Stability (Lower is Better)'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# Growth rates
for col, color in [('YoY Growth (%)', '#1f77b4'), ('3Y CAGR (%)', '#ff7f0e'), ('5Y CAGR (%)', '#2ca02c')]:
    fig.add_trace(
        go.Bar(x=growth_df['REIT'], y=growth_df[col], name=col, marker_color=color),
        row=1, col=1
    )

# Volatility
fig.add_trace(
    go.Bar(x=growth_df['REIT'], y=growth_df['Revenue Volatility (%)'], name='Volatility', marker_color='#d62728', showlegend=False),
    row=1, col=2
)

fig.update_layout(height=400, title_text="Revenue Growth & Stability Analysis")
fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Std Dev of QoQ Growth (%)", row=1, col=2)

fig.show()

# Key insights
fastest_growing = growth_df.loc[growth_df['3Y CAGR (%)'].idxmax(), 'REIT']
most_stable = growth_df.loc[growth_df['Revenue Volatility (%)'].idxmin(), 'REIT']

print(f"\n📊 Key Insights:")
print(f"Fastest Growing (3Y CAGR): {fastest_growing}")
print(f"Most Stable Revenue: {most_stable}")


📊 Key Insights:
Fastest Growing (3Y CAGR): CTRE
Most Stable Revenue: SBRA


## 4. Profitability Analysis

Analyze net income, operating income, and profit margins.

In [8]:
# Compare net income trends
fig = go.Figure()

for reit in reits:
    ni_df = reit_data[reit]['netincomeloss']
    quarterly = ni_df[ni_df['fp'].str.startswith('Q')].copy()
    quarterly = quarterly.sort_values('end')
    
    fig.add_trace(go.Scatter(
        x=quarterly['end'],
        y=quarterly['val'] / 1e6,
        mode='lines+markers',
        name=reit.upper(),
        line=dict(width=3, color=colors[reit]),
        marker=dict(size=6)
    ))

fig.update_layout(
    title='Quarterly Net Income Trends',
    xaxis_title='Quarter End Date',
    yaxis_title='Net Income (Millions USD)',
    hovermode='x unified',
    height=500
)

fig.show()

In [9]:
# Calculate profit margins and profitability metrics
profitability_analysis = []

for reit in reits:
    # Get latest TTM revenue
    rev_df = reit_data[reit]['revenues']
    quarterly_rev = rev_df[rev_df['fp'].str.startswith('Q')].copy()
    quarterly_rev = quarterly_rev.sort_values('end', ascending=False)
    
    # Get latest TTM net income
    ni_df = reit_data[reit]['netincomeloss']
    quarterly_ni = ni_df[ni_df['fp'].str.startswith('Q')].copy()
    quarterly_ni = quarterly_ni.sort_values('end', ascending=False)
    
    # Get latest operating income
    oi_df = reit_data[reit]['operatingincomeloss']
    quarterly_oi = oi_df[oi_df['fp'].str.startswith('Q')].copy() if len(oi_df) > 0 else pd.DataFrame()
    
    if len(quarterly_rev) >= 4 and len(quarterly_ni) >= 4:
        ttm_revenue = quarterly_rev.iloc[0:4]['val'].sum()
        ttm_net_income = quarterly_ni.iloc[0:4]['val'].sum()
        
        # Calculate margins
        net_margin = (ttm_net_income / ttm_revenue) * 100
        
        # Operating margin (if available)
        if len(quarterly_oi) >= 4:
            quarterly_oi = quarterly_oi.sort_values('end', ascending=False)
            ttm_operating_income = quarterly_oi.iloc[0:4]['val'].sum()
            operating_margin = (ttm_operating_income / ttm_revenue) * 100
        else:
            operating_margin = None
        
        # Get equity for ROE calculation
        equity_df = reit_data[reit]['stockholdersequity']
        if len(equity_df) > 0:
            latest_equity = equity_df.iloc[0]['val']
            roe = (ttm_net_income / latest_equity) * 100
        else:
            roe = None
        
        # Get assets for ROA calculation
        assets_df = reit_data[reit]['assets']
        if len(assets_df) > 0:
            latest_assets = assets_df.iloc[0]['val']
            roa = (ttm_net_income / latest_assets) * 100
        else:
            roa = None
        
        profitability_analysis.append({
            'REIT': reit.upper(),
            'TTM Net Income ($M)': ttm_net_income / 1e6,
            'Net Margin (%)': net_margin,
            'Operating Margin (%)': operating_margin,
            'ROE (%)': roe,
            'ROA (%)': roa
        })

profit_df = pd.DataFrame(profitability_analysis)
profit_df

,REIT,TTM Net Income ($M),Net Margin (%),Operating Margin (%),ROE (%),ROA (%)
0,OHI,672.49,48.22,49.20,13.48,6.38
1,CTRE,341.64,54.44,NaN,10.35,7.34
2,SBRA,241.48,26.15,NaN,8.91,4.53


In [10]:
# Visualize profitability metrics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Net Margin', 'Operating Margin', 'Return on Equity (ROE)', 'Return on Assets (ROA)'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}], [{'type': 'bar'}, {'type': 'bar'}]]
)

# Net Margin
fig.add_trace(
    go.Bar(x=profit_df['REIT'], y=profit_df['Net Margin (%)'], marker_color='#1f77b4', showlegend=False),
    row=1, col=1
)

# Operating Margin
fig.add_trace(
    go.Bar(x=profit_df['REIT'], y=profit_df['Operating Margin (%)'], marker_color='#ff7f0e', showlegend=False),
    row=1, col=2
)

# ROE
fig.add_trace(
    go.Bar(x=profit_df['REIT'], y=profit_df['ROE (%)'], marker_color='#2ca02c', showlegend=False),
    row=2, col=1
)

# ROA
fig.add_trace(
    go.Bar(x=profit_df['REIT'], y=profit_df['ROA (%)'], marker_color='#d62728', showlegend=False),
    row=2, col=2
)

fig.update_layout(height=600, title_text="Profitability Metrics Comparison")
fig.update_yaxes(title_text="%", row=1, col=1)
fig.update_yaxes(title_text="%", row=1, col=2)
fig.update_yaxes(title_text="%", row=2, col=1)
fig.update_yaxes(title_text="%", row=2, col=2)

fig.show()

# Key insights
highest_margin = profit_df.loc[profit_df['Net Margin (%)'].idxmax(), 'REIT']
highest_roe = profit_df.loc[profit_df['ROE (%)'].idxmax(), 'REIT']

print(f"\n📊 Key Insights:")
print(f"Highest Net Margin: {highest_margin} ({profit_df[profit_df['REIT']==highest_margin]['Net Margin (%)'].values[0]:.1f}%)")
print(f"Highest ROE: {highest_roe} ({profit_df[profit_df['REIT']==highest_roe]['ROE (%)'].values[0]:.1f}%)")


📊 Key Insights:
Highest Net Margin: CTRE (54.4%)
Highest ROE: OHI (13.5%)


## 5. Balance Sheet Health

Analyze leverage, debt levels, and equity strength.

In [11]:
# Calculate balance sheet metrics
balance_sheet_analysis = []

for reit in reits:
    # Get latest balance sheet items
    assets_df = reit_data[reit]['assets']
    liabilities_df = reit_data[reit]['liabilities']
    equity_df = reit_data[reit]['stockholdersequity']
    debt_df = reit_data[reit]['longtermdebt']
    
    if len(assets_df) > 0 and len(liabilities_df) > 0 and len(equity_df) > 0:
        latest_assets = assets_df.iloc[0]['val']
        latest_liabilities = liabilities_df.iloc[0]['val']
        latest_equity = equity_df.iloc[0]['val']
        
        if len(debt_df) > 0:
            latest_debt = debt_df.iloc[0]['val']
        else:
            latest_debt = None
        
        # Calculate ratios
        debt_to_equity = (latest_debt / latest_equity) if latest_debt else None
        debt_to_assets = (latest_debt / latest_assets) if latest_debt else None
        equity_to_assets = (latest_equity / latest_assets)
        
        balance_sheet_analysis.append({
            'REIT': reit.upper(),
            'Total Assets ($B)': latest_assets / 1e9,
            'Total Liabilities ($B)': latest_liabilities / 1e9,
            'Stockholders Equity ($B)': latest_equity / 1e9,
            'Long-term Debt ($B)': latest_debt / 1e9 if latest_debt else None,
            'Debt/Equity Ratio': debt_to_equity,
            'Debt/Assets (%)': debt_to_assets * 100 if debt_to_assets else None,
            'Equity/Assets (%)': equity_to_assets * 100
        })

balance_df = pd.DataFrame(balance_sheet_analysis)
balance_df

,REIT,Total Assets ($B),Total Liabilities ($B),Stockholders Equity ($B),Long-term Debt ($B),Debt/Equity Ratio,Debt/Assets (%),Equity/Assets (%)
0,OHI,10.55,5.36,4.99,5.00,1.00,47.41,47.30
1,CTRE,4.66,1.33,3.30,1.16,0.35,24.81,70.88
2,SBRA,5.33,2.62,2.71,2.48,0.91,46.54,50.89


In [12]:
# Visualize capital structure
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Capital Structure ($B)', 'Debt/Equity Ratio', 'Leverage (Debt/Assets)'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]]
)

# Stacked bar chart for capital structure
fig.add_trace(
    go.Bar(x=balance_df['REIT'], y=balance_df['Long-term Debt ($B)'], name='Debt', marker_color='#d62728'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=balance_df['REIT'], y=balance_df['Stockholders Equity ($B)'], name='Equity', marker_color='#2ca02c'),
    row=1, col=1
)

# Debt/Equity ratio
fig.add_trace(
    go.Bar(x=balance_df['REIT'], y=balance_df['Debt/Equity Ratio'], marker_color='#ff7f0e', showlegend=False),
    row=1, col=2
)

# Debt/Assets percentage
fig.add_trace(
    go.Bar(x=balance_df['REIT'], y=balance_df['Debt/Assets (%)'], marker_color='#1f77b4', showlegend=False),
    row=1, col=3
)

fig.update_layout(height=400, title_text="Balance Sheet & Leverage Analysis", barmode='stack')
fig.update_yaxes(title_text="Billions USD", row=1, col=1)
fig.update_yaxes(title_text="Ratio", row=1, col=2)
fig.update_yaxes(title_text="%", row=1, col=3)

fig.show()

# Key insights
lowest_leverage = balance_df.loc[balance_df['Debt/Equity Ratio'].idxmin(), 'REIT']
highest_equity = balance_df.loc[balance_df['Stockholders Equity ($B)'].idxmax(), 'REIT']

print(f"\n📊 Key Insights:")
print(f"Lowest Leverage (D/E): {lowest_leverage}")
print(f"Strongest Equity Base: {highest_equity}")
print(f"\nLeverage Comparison:")
for _, row in balance_df.iterrows():
    print(f"  {row['REIT']}: {row['Debt/Equity Ratio']:.2f}x D/E, {row['Debt/Assets (%)']:.1f}% Debt/Assets")


📊 Key Insights:
Lowest Leverage (D/E): CTRE
Strongest Equity Base: OHI

Leverage Comparison:
  OHI: 1.00x D/E, 47.4% Debt/Assets
  CTRE: 0.35x D/E, 24.8% Debt/Assets
  SBRA: 0.91x D/E, 46.5% Debt/Assets


In [13]:
# Debt trend over time
fig = go.Figure()

for reit in reits:
    debt_df = reit_data[reit]['longtermdebt']
    if len(debt_df) > 0:
        debt_df = debt_df.sort_values('end')
        
        fig.add_trace(go.Scatter(
            x=debt_df['end'],
            y=debt_df['val'] / 1e9,
            mode='lines+markers',
            name=reit.upper(),
            line=dict(width=3, color=colors[reit]),
            marker=dict(size=6)
        ))

fig.update_layout(
    title='Long-term Debt Trends',
    xaxis_title='Date',
    yaxis_title='Long-term Debt (Billions USD)',
    hovermode='x unified',
    height=500
)

fig.show()

## 6. Comprehensive REIT Scorecard

Combine all metrics into a single comparison scorecard.

In [14]:
# Create comprehensive scorecard
scorecard = pd.DataFrame({
    'Metric': [
        '--- SIZE METRICS ---',
        'Total Assets ($B)',
        'Real Estate ($B)',
        'TTM Revenue ($M)',
        '',
        '--- GROWTH METRICS ---',
        'YoY Revenue Growth (%)',
        '3-Year Revenue CAGR (%)',
        '5-Year Revenue CAGR (%)',
        'Revenue Volatility (%) *',
        '',
        '--- PROFITABILITY ---',
        'TTM Net Income ($M)',
        'Net Margin (%)',
        'Return on Equity (%)',
        'Return on Assets (%)',
        '',
        '--- BALANCE SHEET ---',
        'Stockholders Equity ($B)',
        'Long-term Debt ($B)',
        'Debt/Equity Ratio',
        'Debt/Assets (%) *'
    ]
})

# Populate with data
for reit in reits:
    reit_upper = reit.upper()
    
    scorecard[reit_upper] = [
        '',  # Section header
        f"{size_df[size_df['REIT']==reit_upper]['Total Assets ($B)'].values[0]:.2f}",
        f"{size_df[size_df['REIT']==reit_upper]['Real Estate ($B)'].values[0]:.2f}",
        f"{size_df[size_df['REIT']==reit_upper]['TTM Revenue ($M)'].values[0]:.1f}",
        '',
        '',  # Section header
        f"{growth_df[growth_df['REIT']==reit_upper]['YoY Growth (%)'].values[0]:.2f}",
        f"{growth_df[growth_df['REIT']==reit_upper]['3Y CAGR (%)'].values[0]:.2f}",
        f"{growth_df[growth_df['REIT']==reit_upper]['5Y CAGR (%)'].values[0]:.2f}" if growth_df[growth_df['REIT']==reit_upper]['5Y CAGR (%)'].values[0] else 'N/A',
        f"{growth_df[growth_df['REIT']==reit_upper]['Revenue Volatility (%)'].values[0]:.2f}",
        '',
        '',  # Section header
        f"{profit_df[profit_df['REIT']==reit_upper]['TTM Net Income ($M)'].values[0]:.1f}",
        f"{profit_df[profit_df['REIT']==reit_upper]['Net Margin (%)'].values[0]:.2f}",
        f"{profit_df[profit_df['REIT']==reit_upper]['ROE (%)'].values[0]:.2f}" if profit_df[profit_df['REIT']==reit_upper]['ROE (%)'].values[0] else 'N/A',
        f"{profit_df[profit_df['REIT']==reit_upper]['ROA (%)'].values[0]:.2f}" if profit_df[profit_df['REIT']==reit_upper]['ROA (%)'].values[0] else 'N/A',
        '',
        '',  # Section header
        f"{balance_df[balance_df['REIT']==reit_upper]['Stockholders Equity ($B)'].values[0]:.2f}",
        f"{balance_df[balance_df['REIT']==reit_upper]['Long-term Debt ($B)'].values[0]:.2f}" if balance_df[balance_df['REIT']==reit_upper]['Long-term Debt ($B)'].values[0] else 'N/A',
        f"{balance_df[balance_df['REIT']==reit_upper]['Debt/Equity Ratio'].values[0]:.2f}" if balance_df[balance_df['REIT']==reit_upper]['Debt/Equity Ratio'].values[0] else 'N/A',
        f"{balance_df[balance_df['REIT']==reit_upper]['Debt/Assets (%)'].values[0]:.1f}" if balance_df[balance_df['REIT']==reit_upper]['Debt/Assets (%)'].values[0] else 'N/A',
    ]

print("\n" + "="*80)
print("COMPREHENSIVE REIT SCORECARD")
print("="*80)
print("* Lower is better for these metrics\n")
scorecard


COMPREHENSIVE REIT SCORECARD
* Lower is better for these metrics



,Metric,OHI,CTRE,SBRA
0,--- SIZE METRICS ---,,,
1,Total Assets ($B),10.55,4.66,5.33
2,Real Estate ($B),6.80,3.26,4.48
3,TTM Revenue ($M),1394.6,495.6,923.4
4,,,,
5,--- GROWTH METRICS ---,,,
6,YoY Revenue Growth (%),-21.37,80.78,-24.07
7,3-Year Revenue CAGR (%),-13.18,24.99,-14.01
8,5-Year Revenue CAGR (%),-0.51,16.25,0.45
9,Revenue Volatility (%) *,84.84,80.33,73.36


## 7. Key Findings & Investment Implications

Summary of the comparative analysis.

In [15]:
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

print("\n🏆 LEADERS BY CATEGORY:\n")

categories = {
    'Size (Assets)': size_df.loc[size_df['Total Assets ($B)'].idxmax(), 'REIT'],
    'Revenue Scale': size_df.loc[size_df['TTM Revenue ($M)'].idxmax(), 'REIT'],
    'Growth (3Y CAGR)': growth_df.loc[growth_df['3Y CAGR (%)'].idxmax(), 'REIT'],
    'Revenue Stability': growth_df.loc[growth_df['Revenue Volatility (%)'].idxmin(), 'REIT'],
    'Profitability (Net Margin)': profit_df.loc[profit_df['Net Margin (%)'].idxmax(), 'REIT'],
    'Return on Equity': profit_df.loc[profit_df['ROE (%)'].idxmax(), 'REIT'],
    'Balance Sheet Strength (Low Leverage)': balance_df.loc[balance_df['Debt/Equity Ratio'].idxmin(), 'REIT'],
    'Equity Strength': balance_df.loc[balance_df['Stockholders Equity ($B)'].idxmax(), 'REIT']
}

for category, leader in categories.items():
    print(f"  {category:.<40} {leader}")

print("\n" + "="*80)
print("COMPARATIVE PROFILES")
print("="*80)

print("\n📊 OHI (Omega Healthcare):")
print("  - Largest REIT by assets and revenue")
print("  - Market leader position")
print("  - Mature, stable cash flows")
print("  - Lower growth but higher stability")

print("\n📊 CTRE (CareTrust):")
print("  - Smallest of the three by assets")
print("  - Higher growth profile")
print("  - More nimble and potentially more upside")
print("  - Balance sheet considerations")

print("\n📊 SBRA (Sabra Healthcare):")
print("  - Middle-market position")
print("  - Balanced growth and stability")
print("  - Diversified portfolio approach")
print("  - Moderate risk/return profile")

print("\n" + "="*80)
print("NEXT STEPS FOR DEEPER ANALYSIS")
print("="*80)
print("""
1. Portfolio Quality Analysis
   → Match properties to CMS quality ratings
   → Calculate weighted average portfolio quality scores
   → Identify concentration risks by geography and tenant

2. Integrate BLS Economic Data
   → Apply PPI growth forecasts to revenue projections
   → Assess labor cost pressure on tenant margins
   → Build forward-looking NOI estimates

3. Build DCF Valuation Models
   → Project 5-year cash flows using BLS indicators
   → Risk-adjust discount rates based on portfolio quality
   → Generate intrinsic value ranges for each REIT

4. Scenario Analysis
   → Base case: Current trends continue
   → Bull case: PPI acceleration, wage moderation
   → Bear case: Wage inflation, PPI stagnation
""")


KEY FINDINGS

🏆 LEADERS BY CATEGORY:

  Size (Assets)........................... OHI
  Revenue Scale........................... OHI
  Growth (3Y CAGR)........................ CTRE
  Revenue Stability....................... SBRA
  Profitability (Net Margin).............. CTRE
  Return on Equity........................ OHI
  Balance Sheet Strength (Low Leverage)... CTRE
  Equity Strength......................... OHI

COMPARATIVE PROFILES

📊 OHI (Omega Healthcare):
  - Largest REIT by assets and revenue
  - Market leader position
  - Mature, stable cash flows
  - Lower growth but higher stability

📊 CTRE (CareTrust):
  - Smallest of the three by assets
  - Higher growth profile
  - More nimble and potentially more upside
  - Balance sheet considerations

📊 SBRA (Sabra Healthcare):
  - Middle-market position
  - Balanced growth and stability
  - Diversified portfolio approach
  - Moderate risk/return profile

NEXT STEPS FOR DEEPER ANALYSIS

1. Portfolio Quality Analysis
   → Match propert

## 8. Export Results for Further Analysis

In [ ]:
# Save comprehensive scorecard
output_path = Path('../../reports')
output_path.mkdir(exist_ok=True)

scorecard.to_csv(output_path / 'reit_comparison_scorecard.csv', index=False)
print(f"✅ Scorecard saved to {output_path / 'reit_comparison_scorecard.csv'}")

# Save individual metric tables
size_df.to_csv(output_path / 'reit_size_metrics.csv', index=False)
growth_df.to_csv(output_path / 'reit_growth_metrics.csv', index=False)
profit_df.to_csv(output_path / 'reit_profitability_metrics.csv', index=False)
balance_df.to_csv(output_path / 'reit_balance_sheet_metrics.csv', index=False)

print("\n✅ All analysis tables exported to reports/ directory")